# MongoDB Cone Searches with HEALPix

In this notebook, we will go over how you can implement simple astronomical cone searches using HEALPix in a NoSQL database. We'll use MongoDB and the `BrownDwarf` collection we create in the first tutorial. Now, MongoDB *does* have spatial indexing capabilities, but we'll go over those in a future post. In this case, we will assume we don't know or can't use these and thus build our own way to quickly search for objects using sky coordinates.

## HEALPix

HEALPix stands for Hierarchical Equal Area isoLatitude Pixelisation and is an algorithm for dividing up the sky in equal-area pixels. This is in contrast to other algorithms like HTM, Hierarchical Triangular Mesh, which divide up the sky in triangles of equal-length. We'll stick to HEALPix since it's used as the basis for several IVOA standards like MOC and HiPS. There are two main Python packages for working with HEALPix- `healpy` and `astropy-healpix`. For simplicity, I'll only use the later code here. For more details, refer to the documentation.

In [16]:
from astropy_healpix import HEALPix, pixel_resolution_to_nside
from astropy.coordinates import ICRS, SkyCoord
from astropy import units as u

# nside required for chosen resolution
resolution = 10 * u.arcsec
nside = pixel_resolution_to_nside(resolution, round='up')
print('NSide: ', nside)

# HEALPix object with that resolution
hp = HEALPix(nside=nside, order='nested', frame=ICRS())

print(hp.pixel_resolution.to(u.arcsec))
print('Number of pixels: ', hp.npix)

NSide:  32768
6.441537022157636 arcsec
Number of pixels:  12884901888


Note that HEALPix values could be equatorial, galactic, ecliptic and their numbering can follow two schemes- RING or NESTED. For consistency with other standards (namely MOC), we'll chose ICRS equatorial and NESTED. In the code above, I create a HEALPix object with pixel resolution that's at least 10 arcseconds or better. That does mean that objects closer than my resolution will be inside the same pixel. For practical applications, you'll probably want a resolution comparable to your PSF or instrument pixel size.

The nside is just a measure of resolution- it's the number of pixels along an edge of the highest level HEALPix pixels. The larger it is, the smaller the pixels, and the more precise your representation. At the chosen precision level, you can see that there are over 12 billion pixels, each represented with an integer value. For example, here's what pixel 42 at this resolution level correponds to in real-sky coordinates:

In [27]:
lon, lat = hp.healpix_to_lonlat([42])
print(lon, lat)
print(lon.degree, lat.degree)

[0.78523rad] [0.00016276rad]
[44.99038696] [0.00932548]


In [13]:
coords = hp.healpix_to_skycoord([42])
print(coords)

<SkyCoord (ICRS): (ra, dec) in deg
    [(44.99038696, 0.00932548)]>


In [25]:
# Now, for the reverse
coords = SkyCoord(ra=34*u.deg, dec=-23*u.deg)
print(hp.skycoord_to_healpix(coords))

9542850888


With this you can probably start putting things togeter- if we can store object locations as integers, then it's just a matter of comparing lists of integers to see if my search area intersects any physical objects. Let's have a look at one more built-in method of `astropy-healpix`:

In [28]:
coords = SkyCoord(ra=34*u.deg, dec=-23*u.deg)
hp_to_search = hp.cone_search_skycoord(coords, radius=5 * u.arcmin)
print(len(hp_to_search))
print(hp_to_search[0:10])

6988
[9542850888 9542850889 9542850891 9542850890 9542850847 9542850845
 9542850839 9542850882 9542850883 9542850886]


This cone search function provides all HEALPix pixels that are in the specified area. Note that because of our chosen resolution level, as well as the size of the cone search, there can be a lot of pixels returned. In this example, nearly 7000 integers were returned. If you have an arbitrary shape, perhaps one defined by an STC-S polygon, it's not too hard to code up something that will return all pixels inside your shape.

## MongoDB Updates

Now that we have prototyped how to create HEALPix values for target RA/Dec and for cone searches, let's update the `BrownDwarf` collection we previously generated in the first tutorial to include these values.

## Cone Search Function

Finally, let's create a utility wrapper to perform these cone searches against the database without having the user calculate HEALPix values manually.